In [275]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


In [276]:
training_data = pd.read_csv('train.csv')
testing_data = pd.read_csv('test.csv')

In [277]:
training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [278]:
features_columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']
target_column = ['Survived']

In [279]:
def transform_sex(x):
    if x == 'male':
        return 0
    else:
        return 1

def stringify(x):
    if pd.isnull(x):
        return 'NaN'
    else:
        return x
    
    
# for training data
training_data['Sex'] = training_data['Sex'].apply(transform_sex)
training_data['Cabin'] = training_data['Cabin'].apply(stringify)
training_data['Embarked'] = training_data['Embarked'].apply(stringify)
Encoder = LabelEncoder()
training_data['Cabin'] = Encoder.fit_transform(training_data['Cabin'])
training_data['Embarked'] = Encoder.fit_transform(training_data['Embarked'])

# for testing data
testing_data['Sex'] = testing_data['Sex'].apply(transform_sex)
testing_data['Cabin'] = testing_data['Cabin'].apply(stringify)
testing_data['Embarked'] = testing_data['Embarked'].apply(stringify)
Encoder = LabelEncoder()
testing_data['Cabin'] = Encoder.fit_transform(testing_data['Cabin'])
testing_data['Embarked'] = Encoder.fit_transform(testing_data['Embarked'])

In [280]:
X = training_data[features_columns]
y = training_data[target_column]

X_test = testing_data[features_columns]

In [281]:
print(X.isna().sum())
print(X_test.isna().sum())

print(X.head())

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin         0
Embarked      0
dtype: int64
Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Cabin        0
Embarked     0
dtype: int64
   Pclass  Sex   Age  SibSp  Parch     Fare  Cabin  Embarked
0       3    0  22.0      1      0   7.2500    146         3
1       1    1  38.0      1      0  71.2833     81         0
2       3    1  26.0      0      0   7.9250    146         3
3       1    1  35.0      1      0  53.1000     55         3
4       3    0  35.0      0      0   8.0500    146         3


In [282]:
male_mean = X[X['Sex'] == 0]['Age'].mean()
female_mean = X[X['Sex'] == 1]['Age'].mean()
fare_mean = X['Fare'].mean()

In [283]:
print(male_mean)
print(female_mean)

def meanify(row):
    if row['Sex'] == 0 and pd.isnull(row['Age']) == True:
        return male_mean
    elif row['Sex'] == 1 and pd.isnull(row['Age']) == True:
        return female_mean
    else:
        return row['Age']
    
def meanify_fare(row):
    if pd.isnull(row['Fare']):
        return fare_mean
    return row['Fare']

X['Age'] = X.apply(meanify, axis = 1)
X_test['Age'] = X_test.apply(meanify, axis = 1)
X_test['Fare'] = X_test.apply(meanify_fare, axis = 1)

30.72664459161148
27.915708812260537


C:\Users\Abhishek Sharma\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Abhishek Sharma\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Abhishek Sharma\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [284]:
print(X.head())
print(y.head())

   Pclass  Sex   Age  SibSp  Parch     Fare  Cabin  Embarked
0       3    0  22.0      1      0   7.2500    146         3
1       1    1  38.0      1      0  71.2833     81         0
2       3    1  26.0      0      0   7.9250    146         3
3       1    1  35.0      1      0  53.1000     55         3
4       3    0  35.0      0      0   8.0500    146         3
   Survived
0         0
1         1
2         1
3         1
4         0


In [285]:
clf1 = LogisticRegression()
clf1.fit(X, y)

C:\Users\Abhishek Sharma\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Abhishek Sharma\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [286]:
clf2 = DecisionTreeClassifier()
clf2.fit(X, y)

C:\Users\Abhishek Sharma\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [287]:
clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(8, 2))
clf3.fit(X, y)

C:\Users\Abhishek Sharma\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(8, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [288]:
clf4 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf4.fit(X, y)

C:\Users\Abhishek Sharma\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\utils\validation.py:179: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':
C:\Users\Abhishek Sharma\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [289]:
print(accuracy_score(clf1.predict(X), y))
print(accuracy_score(clf2.predict(X), y))
print(accuracy_score(clf3.predict(X), y))
print(accuracy_score(clf4.predict(X), y))

C:\Users\Abhishek Sharma\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\base.py:291: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)
C:\Users\Abhishek Sharma\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\preprocessing\label.py:749: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` 

0.797979797979798
0.9865319865319865
0.3569023569023569
0.8395061728395061


In [290]:
my_predictions1 = clf1.predict(X_test)
my_predictions2 = clf2.predict(X_test)
my_predictions3 = clf3.predict(X_test)
my_predictions4 = clf4.predict(X_test)

C:\Users\Abhishek Sharma\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\base.py:291: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)
C:\Users\Abhishek Sharma\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\preprocessing\label.py:749: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` 

In [291]:
my_predictions = []

for mp1, mp2, mp3, mp4 in zip(my_predictions1, my_predictions2, my_predictions3, my_predictions4):
    my_predictions.append(1 if mp1 + mp2 + mp3 + mp4 >= 2 else 0)

my_predictions

[0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,


In [292]:
results = pd.DataFrame({'PassengerId': testing_data['PassengerId'], 'Survived': my_predictions})

In [293]:
results.to_csv('submission.csv', index = False)

In [294]:
my_submission = pd.read_csv('submission.csv')